In [ ]:
df = pd.read_csv('./data.csv')

In [ ]:
X = df['text']
y = df[df.columns[2:]].values

In [ ]:
MAX_FEATURES = 200000 
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [ ]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
model.summary()

In [ ]:
history = model.fit(train, epochs=13, validation_data=val)

In [ ]:
def score_comment(comment):
    input_text = vectorizer([comment])
    res=(model.predict(input_text))
    out=''
    l=[]
    a1=(res1 >=0.5).astype(int)
    for i in range(0,len(a1[0])):
        if a1[0][i]==1:
            l.append(i)
    if len(l)!=0:
        out += 'is_toxic : true\n'+'{}: {}\n'.format('toxic_rate between 0 to 6', len(l))
    else:
        out += 'is_toxic : False\n'
    l=[]
    a2=(res2 >=0.5).astype(int)
    for i in range(1,len(a2[0])):
        if a2[0][i]==1:
            l.append(i)
    if res3[0][0]>0.7:
        try:
            c=max(l)
            out += 'is_humor : true\n'+'{}: {}\n'.format('humor_rate between 0 to 3', c)
        except:
            out += 'is_humor : False\n'
    else:
        out += 'is_humor : False\n'
    return(out)

In [ ]:
interface = gr.Interface(fn=score_comment,inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),outputs='text')
interface.launch(share=True)

In [ ]:
S